In [1]:
import typing
from typing import List, Dict, Tuple
import os
import json

import pandas as pd
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from numpy import dot, ndarray
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import table


import nltk
from nltk.corpus import stopwords

# Download the stop words set
nltk.download('stopwords')



# import capturePPMI
from capturePPMI import *

import importlib
import capturePPMI
importlib.reload(capturePPMI)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenyujie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'capturePPMI' from '/Users/chenyujie/Library/CloudStorage/OneDrive-QueenMary,UniversityofLondon/tensorflow-test/capturePPMI.py'>

In [121]:
shifting_words = get_sorted_positive_words('/Users/chenyujie/Library/CloudStorage/OneDrive-QueenMary,UniversityofLondon/tensorflow-test/coefs_with_fns_2023.pkl')

In [2]:
def reading_data(folder_path: str, header: str = 'text', encoding: str = 'ISO-8859-1') -> List[str]:
    all_corpus = []  # List to collect all text data from all files

    # List all files in the given directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it is a file and ends with .csv
        if os.path.isfile(file_path):
            print(f"Processing file: {file_path}")
            df = pd.read_csv(file_path, header='infer', encoding=encoding)

            # Drop rows where the specific column has NaN values
            df = df.dropna(subset=[header])

            # Extract text data from the specified header/column
            corpus = df[header].tolist()

            # Append this file's corpus to the overall corpus list
            all_corpus.extend(corpus)

    # You might want to process the combined corpus further here
    corpus = Corpus(all_corpus)
    return corpus.get_corpus()


In [4]:
# tokenized_corpus = [ reading_data(path) for path in file_paths]
# Do not need to tokenize once more
def load_tokenized_sentences(file_info: List[str], file_dec= "twStream") -> List[List[str]]:
    """
    Load tokenized sentences from JSON files named after their corresponding file information.

    Args:
        file_info (List[str]): List of file information strings.

    Returns:
        List[List[str]]: List of tokenized sentences.
    """
    tokenized_corpus = []
    for info in file_info:
        file_path = f"twitterStream_corpus_tokenzied/{file_dec}_{info}_tokenized_sentences.json"
        with open(file_path, 'r') as f:
            tokenized_sentence = json.load(f)
            tokenized_corpus.append(tokenized_sentence)
    return tokenized_corpus

file_info = ['14','23']
tokenized_corpus = load_tokenized_sentences(file_info)
print(len(tokenized_corpus))

2


In [5]:
model_14 = Word2Vec (sentences=tokenized_corpus[0], vector_size=200, window=10, min_count=30, workers=4)
model_23 = Word2Vec (sentences=tokenized_corpus[1], vector_size=200, window=10, min_count=30, workers=4)


In [ ]:
models = [model_14, model_23]
# normalised all vectors
for model in models:
    model.wv.fill_norms(force=False)

In [ ]:
slut_words = ["nigger","ass","motherfucker","faggot"]   # This four words are absolute aggressive and toxic. Normally they are very dirty


In [ ]:
mean_vectors = [ model.wv.get_mean_vector(keys=slut_words) for model in models ]

In [ ]:
for vector , model in   zip (mean_vectors,models ):
    print ('the most similar words: ',model.wv.most_similar(positive = vector))

the most similar words:  [('faggot', 0.7463623285293579), ('motherfucker', 0.7455682158470154), ('nigger', 0.7319856882095337), ('fag', 0.6741265654563904), ('bastard', 0.6602538228034973), ('loser', 0.6028072834014893), ('cunt', 0.5994704961776733), ('ass', 0.5496298670768738), ('twat', 0.5445891618728638), ('dickhead', 0.5290022492408752)]
the most similar words:  [('faggot', 0.662257969379425), ('motherfucker', 0.5899576544761658), ('boi', 0.5440467000007629), ('k)not', 0.5345109701156616), ('frð\x9f\x98\xadð\x9f\x98\xadð\x9f\x98\xad', 0.5246606469154358), ('but(t', 0.5233163237571716), ('long.ð\x9f\x98\x8eð\x9fª¢', 0.5199124217033386), ('whalesub', 0.5156594514846802), ('egregious', 0.5152111053466797), ('joi', 0.5097001791000366)]


In [122]:
shifting_words = get_sorted_positive_words('coefs_with_fns_2023.pkl')


In [123]:
# remove all words do not appear in  models:
for model in models:
    words = model.wv.index_to_key
    # Filter out words from all_words that are not in keep_words
    shifting_words = [word for word in shifting_words if word in words]

In [90]:
print (len( shifting_words))


1657


In [91]:
shifting_words_vector = find_word_vectors(shifting_words,  models)


In [92]:
def find_sim(words_vectors: Dict[str, List[np.ndarray]], words: List[str], centre_word_vector: List[np.ndarray]) -> Dict[str, List[float]]:
    """
    Find the similarity of vectors of words to corresponding center word vectors.

    Args:
    words_vectors (Dict[str, List[np.ndarray]]): Dictionary mapping words to lists of their vectors.
    words (List[str]): List of words to find similarities for.
    centre_word_vector (List[np.ndarray]): List of central word vectors, one for each word.

    Returns:
    Dict[str, List[float]]: Dictionary mapping words to their similarity scores with the center word vectors.
    """
    similarity_scores = {}
    for word in words:
        if word in words_vectors:
            vectors = words_vectors[word]
            word_similarity = [capture_cosine_similarity(vector, centre_word_vector[i]) for i, vector in enumerate(vectors)]
            similarity_scores[word] = word_similarity
        else:
            # Handling cases where the word might not be in the dictionary
            similarity_scores[word] = f'Word {word} not in dictionary'

    return similarity_scores

In [93]:
shifting_words_vector_sim = find_sim (shifting_words_vector, shifting_words,mean_vectors )

In [95]:
gradient_results = {}
for word, scores in shifting_words_vector_sim.items():
    gradient_results[word] = calculate_similarity_gradient(scores)


In [100]:
from prettytable import PrettyTable

# Function to print results in table format
def print_results(title, words, gradient_results, shifting_words_vector_sim):
    print(title)
    table = PrettyTable()
    table.field_names = ["Word","Similarity Scores",'gradient']

    for word in words:
        if gradient_results.get(word, None) is None:
            print (f'word {word} do not appear in this corpus')
            continue
        gradients = gradient_results[word]
        similarities = shifting_words_vector_sim[word]
        table.add_row([word, similarities, gradients])

    print(table)



In [101]:
word2vec_list = ['laden','man']
print_results("Words with  word2vec", word2vec_list, gradient_results, shifting_words_vector_sim)


Words with  word2vec
word laden do not appear in this corpus
+------+----------------------------+---------------+
| Word |     Similarity Scores      |    gradient   |
+------+----------------------------+---------------+
| man  | [0.32061785, -0.089538105] | [-0.41015595] |
+------+----------------------------+---------------+


In [102]:
GloVe_word = ['bush', 'throat', 'barely', 'hate']
print_results("Words with  GloVE", GloVe_word, gradient_results, shifting_words_vector_sim)


Words with  GloVE
+--------+------------------------------+---------------+
|  Word  |      Similarity Scores       |    gradient   |
+--------+------------------------------+---------------+
|  bush  | [-0.27505383, 0.0150557235]  |  [0.29010954] |
| throat | [-0.14605299, 0.0001693749]  |  [0.14622237] |
| barely | [-0.063139334, -0.041959338] | [0.021179996] |
|  hate  |  [0.28308094, -0.026849234]  | [-0.30993018] |
+--------+------------------------------+---------------+


In [103]:
PPMI_word =  ['start', 'grab', 'horrible', 'yo', 'park', 'pant','bush', 'americans', 'sheeple', 'fact']
print_results("Words in PPMI :", PPMI_word, gradient_results, shifting_words_vector_sim)


Words in PPMI :
word sheeple do not appear in this corpus
+-----------+------------------------------+----------------+
|    Word   |      Similarity Scores       |    gradient    |
+-----------+------------------------------+----------------+
|   start   |  [0.12455458, -0.005202164]  | [-0.12975675]  |
|    grab   |  [0.07204639, -0.023576263]  | [-0.09562266]  |
|  horrible |   [0.27354857, 0.0828386]    | [-0.19070998]  |
|     yo    |   [0.33664596, 0.12915976]   |  [-0.2074862]  |
|    park   | [-0.03969823, -0.033297468]  | [0.0064007603] |
|    pant   | [-0.025848394, 0.0031353822] | [0.028983776]  |
|    bush   | [-0.27505383, 0.0150557235]  |  [0.29010954]  |
| americans |  [-0.029892331, 0.1854209]   |  [0.21531323]  |
|    fact   |  [0.043538038, -0.14710741]  | [-0.19064544]  |
+-----------+------------------------------+----------------+


In [104]:
print_results ("All words in Word2vec", shifting_words, gradient_results, shifting_words_vector_sim)


All words in Word2vec
+----------------+--------------------------------+------------------+
|      Word      |       Similarity Scores        |     gradient     |
+----------------+--------------------------------+------------------+
|      azz       |    [-0.03091303, 0.3409314]    |   [0.3718444]    |
|      bye       |     [0.370026, 0.11689459]     |   [-0.2531314]   |
|      aye       |   [0.21347108, 0.030284235]    |  [-0.18318684]   |
|      eyed      |   [-0.18549226, 0.013500183]   |   [0.19899245]   |
|       by       |   [0.20208432, -0.20572498]    |  [-0.40780932]   |
|      nyc       |   [-0.23661666, -0.11488397]   |   [0.12173269]   |
|      wya       |   [-0.04827539, -0.2539074]    |  [-0.20563203]   |
|     tyler      |   [-0.17421012, 0.12350055]    |   [0.29771066]   |
|       ny       |   [-0.21293338, -0.20565629]   |  [0.0072770864]  |
|     lying      |   [-0.092808574, 0.16052796]   |   [0.25333655]   |
|   hypocrisy    |   [-0.35329434, -0.10876011]   |   [

# The ppmi part

In [110]:
# def get_nppmi_matrix (file_path: str):
#     df = pd.read_csv(file_path, header='infer')
#     df = df.dropna(subset=['text'])
#     corpus = df['text'].tolist()
#     test_corpus = Corpus(corpus)
#     test_corpus.counting_occur()
#     test_corpus.calculate_npmi_vectors()
#     return test_corpus.npmi_vectors

def reading_data(folder_path: str, header: str = 'text', encoding: str = 'ISO-8859-1') -> List[str]:
    all_corpus = []  # List to collect all text data from all files

    # List all files in the given directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it is a file and ends with .csv
        if os.path.isfile(file_path):
            print(f"Processing file: {file_path}")
            df = pd.read_csv(file_path, header='infer', encoding=encoding)

            # Drop rows where the specific column has NaN values
            df = df.dropna(subset=[header])

            # Extract text data from the specified header/column
            corpus = df[header].tolist()

            # Append this file's corpus to the overall corpus list
            all_corpus.extend(corpus)

    # You might want to process the combined corpus further here
    test_corpus = Corpus(all_corpus)
    test_corpus.counting_occur()
    test_corpus.calculate_npmi_vectors()
    return test_corpus.npmi_vectors


In [112]:
file_paths= ['/Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2014','/Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023']
nppmi_martix = [reading_data(file_path) for file_path in file_paths]


Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2014/2014_2
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2014/2014_3
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2014/2014_1


Calculating NPMI: 100%|██████████| 2024339/2024339 [00:00<00:00, 13701915.15it/s]


Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023/2023_5.csv
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023/2023_4.csv
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023/2023_1.csv
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023/2023_3.csv
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023/2022_2.csv
Processing file: /Users/chenyujie/Desktop/ClassifiedData/Tiwwet/2023/2022_1.csv


Calculating NPMI: 100%|██████████| 949162/949162 [00:00<00:00, 11963368.23it/s]


In [124]:
slut_words = ["nigger","ass","motherfucker","faggot"] # Using word 'ass' instead of used word 'asshole', the word 'ass' occur more


In [125]:
shifting_words_vector = {}
for word in shifting_words:
    vectors = [None] * 2  # Initialize a list to hold vectors from each matrix
    skip_word = False  # Flag to determine if word should be skipped

    # Iterate over each matrix in nppmi_matrix
    for i, matrix in enumerate(nppmi_martix):
        if word in matrix:
            vectors[i] = matrix[word]
        else:
            skip_word = True  # Skip this word if it's not present in any of the matrices
            break  # Exit the inner loop early if word is not found in the current matrix

    # Only add the word and its vectors to the dictionary if the word wasn't skipped
    if not skip_word:
        shifting_words_vector[word] = vectors


In [126]:
mean_vectors = [None] * 2
for i in range (len (mean_vectors)):
    mean_vectors[i] = normalize_and_find_mean([  nppmi_martix[i][word]  for word in slut_words])

In [127]:
shifting_words_vector_sim = find_sim (shifting_words_vector, shifting_words,mean_vectors )

In [128]:
gradient_results = {}
for word, scores in shifting_words_vector_sim.items():
    gradient_results[word] = calculate_similarity_gradient(scores)


In [129]:
word2vec_list = ['laden','man']
print_results("Words with  word2vec", word2vec_list, gradient_results, shifting_words_vector_sim)


Words with  word2vec
word laden do not appear in this corpus
+------+-------------------------------------------+------------------------+
| Word |             Similarity Scores             |        gradient        |
+------+-------------------------------------------+------------------------+
| man  | [0.38782764668565944, 0.2102618845522029] | [-0.17756576213345654] |
+------+-------------------------------------------+------------------------+


In [131]:
GloVe_word = ['bush', 'throat', 'barely', 'hate']
print_results("Words with  GloVE", GloVe_word, gradient_results, shifting_words_vector_sim)


Words with  GloVE
+--------+----------------------------------------------+-------------------------+
|  Word  |              Similarity Scores               |         gradient        |
+--------+----------------------------------------------+-------------------------+
|  bush  | [0.049243072388040064, 0.009608791616403474] |  [-0.03963428077163659] |
| throat |  [0.12258662043204666, 0.07022094712545518]  | [-0.052365673306591476] |
| barely |  [0.08131304569186375, 0.05243375365928573]  | [-0.028879292032578018] |
|  hate  |  [0.40432271524008956, 0.16660555321066825]  |  [-0.2377171620294213]  |
+--------+----------------------------------------------+-------------------------+


In [132]:
PPMI_word =  ['start', 'grab', 'horrible', 'yo', 'park', 'pant','bush', 'americans', 'sheeple', 'fact']
print_results("Words in PPMI :", PPMI_word, gradient_results, shifting_words_vector_sim)


Words in PPMI :
word sheeple do not appear in this corpus
+-----------+----------------------------------------------+-------------------------+
|    Word   |              Similarity Scores               |         gradient        |
+-----------+----------------------------------------------+-------------------------+
|   start   |  [0.28962185228859627, 0.17706063998552676]  |  [-0.11256121230306951] |
|    grab   |  [0.12239321671522158, 0.06319147397849846]  | [-0.059201742736723115] |
|  horrible |  [0.08752643465717803, 0.04852026409229184]  |  [-0.03900617056488619] |
|     yo    |  [0.35650495624595047, 0.10162236239017289]  |  [-0.2548825938557776]  |
|    park   |  [0.10127741167720976, 0.06223060840158284]  | [-0.039046803275626915] |
|    pant   |  [0.14691369015722613, 0.07108375842925502]  |  [-0.07582993172797112] |
|    bush   | [0.049243072388040064, 0.009608791616403474] |  [-0.03963428077163659] |
| americans |  [0.0589287702385428, 0.06397772134835635]   |  [0.0050489

In [133]:
def print_results_withGrid(title, words, gradient_results, shifting_words_vector_sim):
    print(title)
    table = PrettyTable()
    table.field_names = ["Word","Similarity Scores","gradient"]

    for word in words:
        if gradient_results.get(word, None) is None:
            continue
        gradients = gradient_results[word]
        similarities = shifting_words_vector_sim[word]
        table.add_row([word, similarities,gradients])

    print(table)
print_results_withGrid("All words in PPMI", shifting_words, gradient_results, shifting_words_vector_sim)


All words in Word2vec
+----------------+-----------------------------------------------+---------------------------+
|      Word      |               Similarity Scores               |          gradient         |
+----------------+-----------------------------------------------+---------------------------+
|      azz       |   [0.04065959015570786, 0.02469802731721108]  |  [-0.015961562838496778]  |
|      bye       |   [0.14827852631098987, 0.04099320866054176]  |   [-0.1072853176504481]   |
|      aye       |  [0.10451140829888934, 0.012896803775914248]  |   [-0.0916146045229751]   |
|      eyed      |  [0.03492279446067392, 0.005750938186577069]  |  [-0.029171856274096852]  |
|       by       |    [0.375298919809487, 0.19879699363680817]   |   [-0.17650192617267882]  |
|      nyc       |  [0.06476131789172428, 0.028189276345037862]  |   [-0.03657204154668642]  |
|      wya       |  [0.03358230659985309, 0.003939209985513105]  |  [-0.029643096614339986]  |
|     tyler      |   [0.0773

# The  GloVe Part

In [6]:
!pip install glovpy
from glovpy import GloVe


In [7]:
word_list = ['nut',
 'bullshit',
 'useless',
 'psycho',
 'joke',
 'yo',
 'insane',
 'slap',
 'slut',
 'lie',
 'disgusting',
 'selfish',
 'nasty',
 'tard']

In [8]:
model_14 = GloVe ( vector_size=200, window_size=10, min_count=30)
model_14.train(tokens  = tokenized_corpus[0])
model_23 = GloVe ( vector_size=200, window_size=10, min_count=30)
model_23.train(tokens  = tokenized_corpus[1])

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400000 tokens.6500000 to

40000050000060000070000080000090000010000001100000120000013000001400000150000016000001700000180000019000002000000210000022000002300000240000025000002600000270000028000002900000300000031000003200000330000034000003500000360000037000003800000390000040000004100000420000043000004400000450000046000004700000480000049000005000000510000052000005300000540000055000005600000570000058000005900000600000061000006200000630000064000006500000660000067000006800000690000070000007100000720000073000007400000Processed 7498501 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 lines.1500000 lines.1600000 lines.1700000 lines.1800000 lines.1900000 lines.2000000 lines.2100000 lines.2200000 lines.2300000 lines.2400000 lines.2500000 lines.2600000 lines.2700000 lines.2800000 lines.

Shuffling cooccurrences...


processed 4944106 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.4944106 lines.Merging temp files: processed 4944106 lines.

TRAINING MODEL
Read 4944106 lines.
Initializing parameters...Using random seed 1713821433
done.
vector size: 200
vocab size: 7609
x_max: 100.000000
alpha: 0.750000


Training model...


...

20000030000040000050000060000070000080000090000010000001100000120000013000001400000150000016000001700000180000019000002000000210000022000002300000240000025000002600000270000028000002900000300000031000003200000330000034000003500000Processed 3550455 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 lines.1500000 lines.1600000 lines.1700000 lines.1800000 lines.1900000 lines.2000000 lines.2100000 lines.2200000 lines.2300000 lines.2400000 lines.2500000 lines.Merging cooccurrence files: processed 2532293 lines.

Using random seed 1713821468
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.

Shuffling cooccurrences...


processed 2532293 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.2532293 lines.Merging temp files: processed 2532293 lines.

TRAINING MODEL
Read 2532293 lines.
Initializing parameters...Using random seed 1713821468
done.
vector size: 200
vocab size: 6145
x_max: 100.000000
alpha: 0.750000


Training model...


04/22/24 - 10:31.09PM, iter: 001, cost: 0.052209
04/22/24 - 10:31.09PM, iter: 002, cost: 0.040776
04/22/24 - 10:31.10PM, iter: 003, cost: 0.036411
04/22/24 - 10:31.10PM, iter: 004, cost: 0.034718
04/22/24 - 10:31.11PM, iter: 005, cost: 0.032921
04/22/24 - 10:31.11PM, iter: 006, cost: 0.030878
04/22/24 - 10:31.12PM, iter: 007, cost: 0.028636
04/22/24 - 10:31.13PM, iter: 008, cost: 0.026480
04/22/24 - 10:31.13PM, iter: 009, cost: 0.024463
04/22/24 - 10:31.14PM, iter: 010, cost: 0.022667
04/22/24 - 10:31.14PM, iter: 011, cost: 0.021077
04/22/24 - 10:31.15PM, iter: 012, cost: 0.019694
04/22/24 - 10:31.15PM, iter: 013, cost: 0.018489
04/22/24 - 10:31.16PM, iter: 014, cost: 0.017436
04/22/24 - 10:31.16PM, iter: 015, cost: 0.016488
04/22/24 - 10:31.17PM, iter: 016, cost: 0.015658
04/22/24 - 10:31.17PM, iter: 017, cost: 0.014916
04/22/24 - 10:31.18PM, iter: 018, cost: 0.014229
04/22/24 - 10:31.18PM, iter: 019, cost: 0.013622
04/22/24 - 10:31.19PM, iter: 020, cost: 0.013063
04/22/24 - 10:31.20P

In [134]:
slut_words = ["nigger","ass","motherfucker","faggot"] # Using word 'ass' instead of used word 'asshole', the word 'ass' occur more

In [135]:
models = [model_14, model_23 ]
# normalised all vectors
for model in models:
    model.wv.fill_norms(force=False)

In [136]:
for i, word in enumerate(word_list):
    if word not in model_14.wv.index_to_key or  word not in model_23.wv.index_to_key:
        word_list.remove(word)


In [137]:
mean_vectors = [ model.wv.get_mean_vector(keys=slut_words) for model in models ]

In [138]:
def find_word_vectors(target_words: List[str], models: List[Word2Vec] ) -> Dict[str, Dict[str, np.ndarray]]:
    """
    Retrieve the word vectors for a list of target words across multiple Word2Vec models.

    Args:
        target_words (List[str]): List of words for which vectors are required.
        models (List[Word2Vec]): List of Word2Vec models from which vectors are to be retrieved.

    Returns:
        Dict[str, Dict[str, np.ndarray]]: A dictionary where keys are words and values are dictionaries.
            Each nested dictionary has model identifiers as keys and the corresponding word vectors as values.
    """
    words_similarities = {}
    # Iterate through each target word
    for word in target_words:
        model_vectors = [None]* len(models)
        # Retrieve vectors from each model
        for i, model in enumerate( models):
            try:
                # Get vector for the word from the current model
                vector = model.wv[word]
                model_vectors[i] = vector
            except KeyError:
                # Handle the case where the word is not in the model's vocabulary
                model_vectors[model] = None
        # Store vectors from all models for the current word
        words_similarities[word] = model_vectors

    return words_similarities

In [143]:
shifting_words_vector = find_word_vectors(shifting_words,  models)

shifting_words_vector_sim = find_sim (shifting_words_vector, shifting_words,mean_vectors )
gradient_results = {}
for word, scores in shifting_words_vector_sim.items():
    gradient_results[word] = calculate_similarity_gradient(scores)

In [146]:
word2vec_list = ['laden','man']
print_results("Words with  word2vec", word2vec_list, gradient_results, shifting_words_vector_sim)


Words with  word2vec
word laden do not appear in this corpus
+------+----------------------------+---------------+
| Word |     Similarity Scores      |    gradient   |
+------+----------------------------+---------------+
| man  | [0.32061785, -0.089538105] | [-0.41015595] |
+------+----------------------------+---------------+


In [145]:
PPMI_word = (['start', 'grab', 'horrible', 'yo', 'park', 'pant','bush', 'americans', 'sheeple', 'fact'])
print_results("Words in PPMI :", PPMI_word, gradient_results, shifting_words_vector_sim)


Words in PPMI :
word sheeple do not appear in this corpus
+-----------+------------------------------+----------------+
|    Word   |      Similarity Scores       |    gradient    |
+-----------+------------------------------+----------------+
|   start   |  [0.12455458, -0.005202164]  | [-0.12975675]  |
|    grab   |  [0.07204639, -0.023576263]  | [-0.09562266]  |
|  horrible |   [0.27354857, 0.0828386]    | [-0.19070998]  |
|     yo    |   [0.33664596, 0.12915976]   |  [-0.2074862]  |
|    park   | [-0.03969823, -0.033297468]  | [0.0064007603] |
|    pant   | [-0.025848394, 0.0031353822] | [0.028983776]  |
|    bush   | [-0.27505383, 0.0150557235]  |  [0.29010954]  |
| americans |  [-0.029892331, 0.1854209]   |  [0.21531323]  |
|    fact   |  [0.043538038, -0.14710741]  | [-0.19064544]  |
+-----------+------------------------------+----------------+


In [147]:
GloVe_word = ['bush', 'throat', 'barely', 'hate']
print_results("Words with  GloVE", GloVe_word, gradient_results, shifting_words_vector_sim)


Words with  GloVE
+--------+------------------------------+---------------+
|  Word  |      Similarity Scores       |    gradient   |
+--------+------------------------------+---------------+
|  bush  | [-0.27505383, 0.0150557235]  |  [0.29010954] |
| throat | [-0.14605299, 0.0001693749]  |  [0.14622237] |
| barely | [-0.063139334, -0.041959338] | [0.021179996] |
|  hate  |  [0.28308094, -0.026849234]  | [-0.30993018] |
+--------+------------------------------+---------------+


In [148]:
print_results_withGrid("All words in GloVe", shifting_words, gradient_results, shifting_words_vector_sim)


All words in GloVe
+----------------+--------------------------------+------------------+
|      Word      |       Similarity Scores        |     gradient     |
+----------------+--------------------------------+------------------+
|      azz       |    [-0.03091303, 0.3409314]    |   [0.3718444]    |
|      bye       |     [0.370026, 0.11689459]     |   [-0.2531314]   |
|      aye       |   [0.21347108, 0.030284235]    |  [-0.18318684]   |
|      eyed      |   [-0.18549226, 0.013500183]   |   [0.19899245]   |
|       by       |   [0.20208432, -0.20572498]    |  [-0.40780932]   |
|      nyc       |   [-0.23661666, -0.11488397]   |   [0.12173269]   |
|      wya       |   [-0.04827539, -0.2539074]    |  [-0.20563203]   |
|     tyler      |   [-0.17421012, 0.12350055]    |   [0.29771066]   |
|       ny       |   [-0.21293338, -0.20565629]   |  [0.0072770864]  |
|     lying      |   [-0.092808574, 0.16052796]   |   [0.25333655]   |
|   hypocrisy    |   [-0.35329434, -0.10876011]   |   [0.2